# Dialogue Analysis Agent 테스트

대화 분석 에이전트 (Level 1) 테스트 노트북

In [ ]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

In [ ]:
SAMPLE_STORY = """
"우리가 이곳에서 그를 만날 수 있을까?" 하나가 조심스럽게 물었다.

"오랜만이군, 서진." 이민호가 차갑게 말했다. "아직도 그 낡은 검을 들고 다니는군."

서진은 눈을 똑바로 바라보며 대답했다. "왜 마을을 배신했어, 민호? 우리가 함께 자란 마을이잖아."

이민호는 씁쓸하게 웃었다. "넌 모르지. 마을 장로들이 내 가족에게 무슨 짓을 했는지. 
그들은 내 아버지를..."

하나가 두 사람 사이로 걸어나왔다. "싸우기 전에 대화를 해봐요. 진실을 알아야 해요."

"진실?" 이민호가 비웃었다. "진실은 항상 강자의 편이지."
"""

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/api/callback",
            "analyzed_dialogues": {}, "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

In [ ]:
from app.agents.extraction.dialogue import dialogue_analysis_node

async def test_dialogue_analysis():
    print("💬 Dialogue Analysis 테스트...")
    state = create_base_state()
    return await dialogue_analysis_node(state)

result = run_async(test_dialogue_analysis())
print(json.dumps(result, ensure_ascii=False, indent=2))